In [47]:
import os
import warnings
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
#os.environ['CUDA_LAUNCH_BLOCKING'] = str(1)
#os.environ["TORCH_USE_CUDA_DSA"]= str(0)
warnings.filterwarnings('ignore') 


In [48]:
import copy
import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.utils.data import Dataset
import torch.optim as optim
from torch.autograd import Variable
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import math
from collections import OrderedDict
import random
from torchsummary import summary
from torchvision import transforms
from torch.utils.data import Dataset
import sys
import torch
import numpy as np
from tqdm import trange
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import mne
from sklearn.preprocessing import StandardScaler
import logging
from torch.nn.utils.rnn import pack_sequence, pad_packed_sequence
import statistics
import torch.optim.lr_scheduler as lr_scheduler

In [49]:
logging.getLogger('mne').setLevel(logging.WARNING)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
import import_ipynb
#from Model import net
from Dataloader2 import EEGDataset

In [50]:
def Accuracy(y_pred, y, train_count):
    max_values, _ = torch.max(y_pred, dim=1, keepdim=True)
    mask = y_pred == max_values
    y_pred = mask.int()
    correct_num = torch.sum(torch.all(torch.eq(y, y_pred), dim=1)).item()
    accuracy = correct_num / train_count
    return accuracy

In [51]:
class net(nn.Module):
    def __init__(self, T, C, input_size, hidden_size, num_layers, spatial_num, dropout, pool):
        super(net, self).__init__()
        
        self.T = T
        self.C = C
        self.spatial_num = spatial_num
        self.dropout = dropout
        self.pool = pool

        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        self.cell_count = self.T // self.input_size

        self.fcn_in = (spatial_num * self.hidden_size)

        self._lstm = nn.LSTM(self.input_size, self.hidden_size, self.num_layers, batch_first=True)

        self.lstm = nn.ModuleList([self._lstm for i in range(self.C)])

        self.cnn_block = nn.Sequential(nn.Conv2d(1, self.spatial_num, (self.C, 1)),
                                       nn.BatchNorm2d(self.spatial_num),
                                       nn.ELU(),



                                       nn.Dropout(self.dropout))

        
        self.fcn = nn.Sequential(nn.Linear(self.fcn_in, 1024), 
                                 nn.ReLU(),
                                 nn.Linear(1024, 16),
                                 nn.ReLU(),
                                 nn.Linear(16, 4))

        #self.fcn = nn.Linear(self.fcn_in, 4)
        self.results = nn.Softmax(dim=1)
    def forward(self, x):
        x = x.reshape(-1, 1, 19, 3000)
        self.N = x.shape[0]
        x = x.reshape(self.N, self.C, self.cell_count, self.input_size)
        _x = None

        for index, cell in enumerate(self.lstm):
            cell_out, _ = cell(x[:, index, :, :], None)
            last_layer_out = cell_out[:, -1, :]
            
            last_layer_out = last_layer_out.unsqueeze(0)
            if _x is None:
                _x = last_layer_out
            else:
                _x = torch.cat((_x, last_layer_out), dim=0)
            

        x = _x.permute(1, 0, 2).unsqueeze(1)

        x = self.cnn_block(x)


        x = x.reshape(self.N, -1)

        x = self.fcn(x)
        x = self.results(x)

        return x

In [52]:
MNE_Data = EEGDataset(root_dir=r"D:\TEST MNE")

In [53]:
batch_size = 1
train_dataloader = DataLoader(MNE_Data, batch_size=batch_size, shuffle=False)

In [54]:
model = net(T = 3000, C = 19, input_size = 300, hidden_size = 30, num_layers=1, spatial_num= 1, dropout=0.5, pool=1).to(device)

In [55]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = lr_scheduler.ExponentialLR(optimizer, gamma=0.95)
epochs = 60

In [ ]:
for epoch in trange(epochs):
        model.train()
        running_loss = 0.0
        correct_num = 0
        log = []
        for index, data in enumerate(train_dataloader):
            
            x, y = data
            y = y.to(torch.float64)
            x = x.reshape(-1, 1, 19, 3000).float()
            y = F.one_hot(torch.tensor(torch.tensor([y.item()]).to(torch.int64)), num_classes=4).expand(x.shape[0], -1).float()
            train_count = x.shape[0]
            x, y = x.to(device), y.to(device)
            y_pred = model(x)
            loss = criterion(y_pred, y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            scheduler.step()
            print(Accuracy(y_pred, y, train_count))
            log.append(Accuracy(y_pred, y, train_count))
            torch.cuda.empty_cache()
        
        print(f"mean accuracy across epoch: {statistics.mean(log)}")
        
        #print(f"Train Accuracy:{correct_num / train_count}")
        

In [ ]:
for epoch in trange(epochs):
        model.train()
        running_loss = 0.0
        correct_num = 0
        log = []
        for index, data in enumerate(train_dataloader):
            
            x, y = data
            y = y.to(torch.float64)
            """
            x = x.reshape(-1, 1, 19, 3000).float()
            y = F.one_hot(torch.tensor(torch.tensor([y.item()]).to(torch.int64)), num_classes=4).expand(x.shape[0], -1).float()
            
            train_count = x.shape[0]
            x, y = x.to(device), y.to(device)
            y_pred = model(x)
            loss = criterion(y_pred, y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            scheduler.step()
            print(y[-1])
            print(Accuracy(y_pred, y, train_count))
            log.append(Accuracy(y_pred, y, train_count))
            torch.cuda.empty_cache()
            
        
        """
        #print(f"mean accuracy across epoch: {statistics.mean(log)}")
        
        #print(f"Train Accuracy:{correct_num / train_count}")

In [64]:
y

tensor([2.], dtype=torch.float64)

In [ ]:
"""
__ = (torch.round(y_pred).to(torch.int64) == y)
preds = np.all(__.cpu().numpy(), axis=1)
correct_num += np.count_nonzero(preds)
"""

'\n__ = (torch.round(y_pred).to(torch.int64) == y)\npreds = np.all(__.cpu().numpy(), axis=1)\ncorrect_num += np.count_nonzero(preds)\n'